# 安徽安庆市项目4月`CMAQ-ISAM`
## 利用`MCIP`生成的`IOAPI`制作`GridMask.nc`

---
*@author: Evan*\
*@date: 2023-04-30*

In [1]:
import numpy as np
import netCDF4 as nc

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

In [2]:
input = nc.Dataset('./input/GRIDCRO2D_2023100.nc','r+')

In [3]:
# METCRO2D
# var_name = {'PRSFC':'Anqing',
#             'USTAR':'Hefei',
#             'WSTAR':'Tongling',
#             'PBL':'Chizhou',
#             'ZRUF':'Jiujiang',
#             'MOLI':'Wuhu',
#             'HFX':'Maanshan'}

# GRIDCRO2D
var_name = {'MSFX2':'Anqing',
            'HT':'Hefei',
            'DLUSE':'Tongling',
            'LWMASK':'Chizhou',
            'PURB':'Jiujiang',}

In [4]:
# 修改变量名
for old_var, new_var in var_name.items():
    variable = input.variables.pop(old_var)  # 取出旧变量
    new_variable = input.createVariable(new_var, variable.datatype, variable.dimensions)  # 根据旧变量创建新变量
    new_variable.setncatts(variable.__dict__)  # 复制旧变量的属性到新变量
    new_variable[:] = variable[:]  # 复制旧变量的值到新变量
    # del input.variables[old_var]

In [5]:
# 修改VAR-LIST
var_list = input.getncattr('VAR-LIST').split()
for i, var in enumerate(var_list):
    if var in var_name:
        var_list[i] = var_name[var].ljust(16)
    else:
        var_list[i] = var_list[i].ljust(16)
input.setncattr('VAR-LIST',''.join(var_list))
input.getncattr('VAR-LIST')

'LAT             LON             Anqing          Hefei           Tongling        Chizhou         Jiujiang        '

In [6]:
import geopandas as gpd
import shapely.geometry as sgeom
from shapely.prepared import prep

def polygon_to_mask(polygon, x, y):
    '''
    Generate a mask array of points falling into the polygon
    '''
    x = np.atleast_1d(x)
    y = np.atleast_1d(y)
    mask = np.zeros(x.shape, dtype=bool)

    # if each point falls into a polygon, without boundaries
    prepared = prep(polygon)
    for index in np.ndindex(x.shape):
        point = sgeom.Point(x[index], y[index])
        if prepared.contains(point):
            mask[index] = True

    return mask

In [7]:
lat = np.squeeze(input['LAT'])
lon = np.squeeze(input['LON'])
lat.shape

(138, 135)

In [8]:
aq1 = input['Hefei']
print(np.mean(aq1))
aq1.shape

156.09528


(1, 1, 138, 135)

In [9]:
%%time

cities = ['Anqing', 'Hefei', 'Tongling', 'Chizhou', 'Jiujiang',]# 'Wuhu', 'Maanshan']

for city in cities:
    shp = gpd.read_file(f'./shp/{city}.shp')
    for i in range(np.size(lat,0)):
        for j in range(np.size(lat,1)):
            if polygon_to_mask(shp.geometry[0], lon[i, j], lat[i, j]) == True:
                input[f'{city}'][:, :, i, j] = 1
            else:
                input[f'{city}'][:, :, i, j] = 0

CPU times: total: 15.4 s
Wall time: 15.5 s


In [10]:
aq2 = input['Hefei']
print(np.mean(aq2))
aq2.shape

0.06596887


(1, 1, 138, 135)

In [13]:
input['LAT'][:]=0
input['LON'][:]=0
print(np.mean(input['LAT']))

0.0


In [21]:
input.close()

---
删除nc中多余的变量(未能成功保存入文件)

In [54]:
output = nc.Dataset('./input/GRIDCRO2D_2023100.nc',mode='r+')
for var in output.variables:
    print(var)

TFLAG
LAT
LON
MSFX2
HT
DLUSE
LWMASK
PURB
Anqing
Hefei
Tongling
Chizhou
Jiujiang


In [55]:
names = ['MSFX2','HT','DLUSE','LWMASK','PURB']

for name in names:
    del output.variables[name]

for var in output.variables:
    print(var)

TFLAG
LAT
LON
Anqing
Hefei
Tongling
Chizhou
Jiujiang


In [57]:
output.close()